In [19]:
import sys
import tweepy,json
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener

# Replace the "None"s by your own credentials
ACCESS_TOKEN = "1173156479786000386-glFQXlJsdRlmmyXuJEVIap326UPTVt"
ACCESS_TOKEN_SECRET = "5ePMGYr8gQmOsofEQ8gx1VZpqLP7Oyl8E7qFmzwbqkwEN"
CONSUMER_KEY = "AJuWhND0dGUdVvTgUCGs087Zo"
CONSUMER_SECRET = "D0h9CIdXmhFPdxUxtHeiQpGu2KpUMUowUtunEEN5OGkk2VjDhe"

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(Listener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweetsstream.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 1000:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

l = Listener()
stream = tweepy.Stream(auth, l)
# This line filters Twitter Streams to capture data by keywords:
stream.filter(track=['apple', 'iphone 12'])

In [21]:
# Import package
import json
import pandas as pd

# String of path to file: tweets_data_path
tweets_data_path='tweetsstream.txt'

# Initialize empty list to store tweets: tweets_data
tweets_data=[]

# Open connection to file
tweets_file = open(tweets_data_path, "r")

# Read in tweets and store in list: tweets_data
for line in tweets_file:
    tweet=json.loads(line)
    tweets_data.append(tweet)
    

# Close connection to file
tweets_file.close()

# Print the keys of the first tweet dict
print(tweets_data[0].keys())

tweets_df=pd.DataFrame(tweets_data,columns=['created_at','text','lang'])
tweets_df['created_at']=pd.to_datetime(tweets_df['created_at'])

print(tweets_df.head())
#print(tweets_df.shape)


dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])
                 created_at  ... lang
0 2020-10-14 09:27:31+00:00  ...   tl
1 2020-10-14 09:27:31+00:00  ...   in
2 2020-10-14 09:27:31+00:00  ...   en
3 2020-10-14 09:27:31+00:00  ...   ja
4 2020-10-14 09:27:31+00:00  ...   th

[5 rows x 3 columns]


In [22]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 9.1MB/s 


In [23]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
sentiment_scores = tweets_df['text'].apply(sid.polarity_scores)
print(sentiment_scores)

0      {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1      {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2      {'neg': 0.249, 'neu': 0.751, 'pos': 0.0, 'comp...
3      {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4      {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                             ...                        
994    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
995    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
996    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
997    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
998    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
Name: text, Length: 999, dtype: object


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [25]:
#sentiment = sentiment_scores.apply(lambda x: x['compound'])
tweets_df['sentiment_score']=sentiment_scores.apply(lambda x: x['compound'])
#sentiment_i=sentiment.resample('1 min').mean()
print(tweets_df)

                   created_at  ... sentiment_score
0   2020-10-14 09:27:31+00:00  ...          0.0000
1   2020-10-14 09:27:31+00:00  ...          0.0000
2   2020-10-14 09:27:31+00:00  ...         -0.8074
3   2020-10-14 09:27:31+00:00  ...          0.0000
4   2020-10-14 09:27:31+00:00  ...          0.0000
..                        ...  ...             ...
994 2020-10-14 09:28:30+00:00  ...          0.0000
995 2020-10-14 09:28:30+00:00  ...          0.0000
996 2020-10-14 09:28:30+00:00  ...          0.0000
997 2020-10-14 09:28:30+00:00  ...          0.0000
998 2020-10-14 09:28:30+00:00  ...          0.0000

[999 rows x 4 columns]
